In [2]:
import json
from pprint import pprint

In [3]:
# Import data

with open('train_new.json') as f:
    data1=json.load(f)

data1.keys()
#pprint(data)

with open('test_new.json') as f:
    data2=json.load(f)

data1.keys()
#pprint(data)

dict_keys(['content', 'date', 'tags', 'title', 'url'])

In [4]:
#Prepare text data

import pandas as pd
set=[]
for k,v in data1['content'].items():
    #print(k)
    #print(v)
    set.append(v)

for k,v in data2['content'].items():
    #print(k)
    #print(v)
    set.append(v)
    
df1=pd.DataFrame(set)
print(df1.head())
print(df1.shape)    
    
df1=pd.DataFrame(set)
df1.head()
df1.shape

                                                   0
0  Approximately 1.6 million model-year 2015-18 F...
1  Approximately 192,000 model-year 2016-18 Toyot...
2  Nobody likes when things don't work the way th...
3  Approximately 19,400 model-year 2012 Toyota Av...
4  Approximately 8,400 model-year 2017-18 Volkswa...
(2815, 1)


(2815, 1)

In [5]:
#df1.columns=['a']

In [6]:
#df1['a']

0       Approximately 1.6 million model-year 2015-18 F...
1       Approximately 192,000 model-year 2016-18 Toyot...
2       Nobody likes when things don't work the way th...
3       Approximately 19,400 model-year 2012 Toyota Av...
4       Approximately 8,400 model-year 2017-18 Volkswa...
5       Approximately 9,000 model-year 2018 Mercedes-B...
6       Approximately 12,800 model-year 2018 Jeep Rene...
7       Approximately 154,000 model-year 2018-19 Dodge...
8       Approximately 90 model-year 2018 Ford Edge and...
9       Approximately 49,000 model-year 2012-15 Ford F...
10      Approximately 470 model-year 2018 Mercedes-Ben...
11      Approximately 3,300 model-year 2018 Genesis G8...
12      Approximately 300 model-year 2019 Subaru Ascen...
13      Approximately 65 model-year 2018 Chrysler Paci...
14      12 model-year 2018 BMW 540d xDrive sedans\n Th...
15      's 2018 Volkswagen Atlas, a long-term test veh...
16      Nobody likes when things don't work the way th...
17      As the

In [7]:
#nltk.download()

In [ ]:
#Pre-process data

In [8]:
import nltk
from nltk.corpus import stopwords
mystopwords=stopwords.words("English") + ['one', 'become', 'get', 'make', 'take', 'recall','said', 'say', 'could', 'nhtsa', 'n\'t', 'may', 'vehicle', 'car']
WNlemma = nltk.WordNetLemmatizer()

def pre_process(text):
    tokens = nltk.word_tokenize(text)
    tokens=[ WNlemma.lemmatize(t.lower()) for t in tokens]
    tokens=[ t for t in tokens if t not in mystopwords]
    tokens = [ t for t in tokens if len(t) >= 3 ]
    text_after_process=" ".join(tokens)
    return(text_after_process)


In [9]:
#Pre-process data

toks_train = df1['a'].apply(pre_process)

In [10]:
#Import library

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline

In [11]:
# Create tfidf matrix
vectorizer = TfidfVectorizer(max_features=2500,
                              stop_words=mystopwords,
                             use_idf=True)
X = vectorizer.fit_transform(toks_train)
X.shape

(2815, 2500)

In [12]:
# Use SVD to reduce dimensions
svd = TruncatedSVD(500)
normalizer = Normalizer(copy=False)
lsa = make_pipeline(svd, normalizer)
X_lsa = lsa.fit_transform(X)


In [13]:
# Check how much variance is explained
explained_variance = svd.explained_variance_ratio_.sum()
print("Explained variance of the SVD step: {}%".format(int(explained_variance * 100)))

Explained variance of the SVD step: 75%


In [14]:
# Cluster analysis

from sklearn.cluster import KMeans
#random_state=4321
km3 = KMeans(n_clusters=5, init='k-means++', max_iter=1000, n_init=1)
%time km3.fit(X_lsa)

#‘k-means++’ : selects initial cluster centers for k-mean clustering in a smart way to speed up convergence.
#n_init : int, default: 10. Number of time the k-means algorithm will be run with different centroid seeds.
#Maximum number of iterations of the k-means algorithm for a single run.

Wall time: 212 ms


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=1000,
    n_clusters=5, n_init=1, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [15]:
from sklearn import metrics

# Silhouette score range (-1 to 1)

print("Silhouette Coefficient for 3 clusters: %0.3f"
      % metrics.silhouette_score(X_lsa, km3.labels_))

Silhouette Coefficient for 3 clusters: 0.024


In [16]:
#Generate terms for each cluster

def print_terms(cm, num):
    original_space_centroids = svd.inverse_transform(cm.cluster_centers_)
    order_centroids = original_space_centroids.argsort()[:, ::-1]
    terms = vectorizer.get_feature_names()
    for i in range(num):
        print("Cluster %d:" % i, end='')
        for ind in order_centroids[i, :10]:
            print(' %s' % terms[ind], end='')
        print()

print_terms(km3, 5)

Cluster 0: brake hyundai safety sonata owner model pedal parking kia fluid
Cluster 1: ford escape lincoln plant fuel fusion door 2013 assembly 2014
Cluster 2: takata inflator honda airbag airbags million side passenger mazda automaker
Cluster 3: model safety fuel owner year chrysler problem seat dealer engine
Cluster 4: toyota lexus pedal acceleration prius million 2010 automaker unintended issue
